## Part 3: Mapspace Exploration with Timeloop

We start with the same architecture and mapping as in Part 2.

<br>
<div class="row">
  <div class="column">
    <img align="left" src="designs/system/figures/arch.png" alt="Full System  Architecture Diagram" style="margin:50px 0px 0px 50px; width:40%">
  </div>
  <div class="column">
    <img  align="left"  src="designs/system/figures/loopnest.png" alt="System Loopnest" style="width:50%">
  </div>
</div>

### Question 1
In this question, we would like you to find the best architecture and associated mapping that has the highest throughput (minimizes the number of cycles) for `layer_shapes/conv2.yaml`. If two architectures result in the same throughput, choose the one that's less energy consuming.

First, inspect the mapping file below and see what double-curly-brace-enclosed parameters are available to set.

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from loaders import *
show_config('designs/system/map.yaml')

# ========================================================================
# Please do not modify this file. If there are double-curly-brace-enclosed
# statements, they are placeholders that should be set from the notebooks.
# ========================================================================
mapping:
- target: DRAM
  type: temporal
  factors: 
  - P=1
  - Q=1
  - R=1
  - S=1
  - N={{DRAM_factor_N}}
  - M={{DRAM_factor_M}}
  - C={{DRAM_factor_C}}
  permutation: [S, R, Q, P, C, M, N] # don't change this

- target: global_buffer
  type: temporal
  factors: 
  - P=1
  - Q=1
  - R=1
  - S=1
  - N={{global_buffer_factor_N}}
  - M={{global_buffer_factor_M}}
  - C={{global_buffer_factor_C}}
  permutation: [S, R, Q, P, C, M, N] # don't change this

- target: PE
  type: spatial  # spatial constraint specification
  factors: 
  - P=1
  - Q=1
  - R=1
  - S=1
  - N=1
  - M={{PE_spatial_factor_M}}
  - C={{PE_spatial_factor_C}}
  permutation: [C, M, R, S, P, Q, N]
  # tells at which index shou

Now, optimize these mappings for a 1x16-PE, 2x8-PE, and 4x4-PE architecture by setting the variables below. To simplify the mapspace, most variables
are set and not able to change. The following constraints are required (and can not be changed by the below variables):

- The mapping agrees with the loop nest provided in the image above.
- Input channels can only be spatially mapped to the rows of the PE array and output channels can only be spatially mapped to the columns of the PE array.
- PE scrachpads only store filter weights.

After optimizing these mappings, please fill out the table below with the energy and delay of each mapping. An example setting is shown below.

**For full credit, all of your optimized mappings must be below 400 uJ and 4,000,000 cycles. Furthermore, at least one mapping must be below 275 uJ.**

In [69]:
config_example = dict( # Do not change this configuration!
    DRAM_factor_N=50,
    DRAM_factor_M=8,
    DRAM_factor_C=4,
    global_buffer_factor_N=1,
    global_buffer_factor_M=1,
    global_buffer_factor_C=1,
    PE_spatial_factor_M=1,
    PE_spatial_factor_C=1,
    scratchpad_factor_N=1,
)

config_optimized_1x16 = dict( # Replace these with your optimized values
	 DRAM_factor_N=50,
     DRAM_factor_M=1,
     DRAM_factor_C=4,
     global_buffer_factor_N=1,
     global_buffer_factor_M=1,
     global_buffer_factor_C=1,
     PE_spatial_factor_M=8,
     PE_spatial_factor_C=1,
     scratchpad_factor_N=1,
########################
#### YOUR CODE HERE ####
########################
)

config_optimized_2x8 = dict( # Replace these with your optimized values
	 DRAM_factor_N=50,
     DRAM_factor_M=1,
     DRAM_factor_C=2,
     global_buffer_factor_N=1,
     global_buffer_factor_M=1,
     global_buffer_factor_C=1,
     PE_spatial_factor_M=8,
     PE_spatial_factor_C=2,
     scratchpad_factor_N=1,
########################
#### YOUR CODE HERE ####
########################
)

config_optimized_4x4 = dict( # Replace these with your optimized values
	 DRAM_factor_N=50,
     DRAM_factor_M=1,
     DRAM_factor_C=1,
     global_buffer_factor_N=1,
     global_buffer_factor_M=2,
     global_buffer_factor_C=1,
     PE_spatial_factor_M=4,
     PE_spatial_factor_C=4,
     scratchpad_factor_N=1,
########################
#### YOUR CODE HERE ####
########################
)

In [70]:
configs = {
    'Example 1x16': {**config_example, "pe_meshX": 1, "pe_meshY": 16},
    'Example 2x8': {**config_example, "pe_meshX": 2, "pe_meshY": 8},
    'Example 4x4': {**config_example, "pe_meshX": 4, "pe_meshY": 4},
    'Optimized 1x16': {**config_optimized_1x16, "pe_meshX": 1, "pe_meshY": 16},
    'Optimized 2x8': {**config_optimized_2x8, "pe_meshX": 2, "pe_meshY": 8},
    'Optimized 4x4': {**config_optimized_4x4, "pe_meshX": 4, "pe_meshY": 4},
}

# Set to None to run all configurations. FOR YOUR FINAL SUBMISSION, MAKE SURE YOU RUN ALL CONFIGURATIONS.
CONFIG_TO_RUN = None # 'Optimized 4x4'

to_run = list(configs.keys()) if CONFIG_TO_RUN is None else [CONFIG_TO_RUN]

# DO NOT CHANGE
THRES = [(float('inf'), float('inf')),
         (float('inf'), float('inf')),
         (float('inf'), float('inf')),
         (4_000_000, 400),
         (4_000_000, 400),
         (4_000_000, 400)]

min_energy = float('inf')
for i, k in enumerate(to_run):
    cycle_thres, energy_thres = THRES[i]
    result = run_timeloop_model(
        configs[k],
        architecture='designs/system/arch.yaml',
        mapping='designs/system/map.yaml',
        problem='layer_shapes/conv2.yaml'
    )
    stats = open('./output_dir/timeloop-model.stats.txt', 'r').read()
    mapping = result.mapping
    if len(to_run) == 1:
        print(stats)

    lines = stats.split('\n')
    energy = float([l for l in lines if 'Energy:' in l][0].split(' ', 2)[1])
    cycles = int([l for l in lines if 'Cycles:' in l][0].split(' ', 1)[1])
    min_energy = min(min_energy, energy)

    if i < 3:
        print(f'{k} --- Latency: {cycles} cycles; Energy: {energy} uJ.')
    else:
        answer( # Don't change this
            question='3.1',
            subquestion=f'{k} --- Latency: {cycles} cycles; Energy: {energy} uJ. Passing?',
            answer=cycles < cycle_thres and energy < energy_thres,
            required_type=bool
        )
    print('')

answer( # Don't change this
    question='3.1',
    subquestion=f'Minimum energy achieved: {min_energy} uJ. Passing?',
    answer=min_energy < 275,
    required_type=bool
)

[INFO] 2025-03-03 22:34:13,489 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


Example 1x16 --- Latency: 31360000 cycles; Energy: 657.11 uJ.

[INFO] 2025-03-03 22:34:14,839 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


Example 2x8 --- Latency: 31360000 cycles; Energy: 657.11 uJ.

[INFO] 2025-03-03 22:34:16,163 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


...

3.1: Optimized 1x16 --- Latency: 3920000 cycles; Energy: 399.0 uJ. Passing?
	True

[INFO] 2025-03-03 22:34:18,940 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


3.1: Optimized 2x8 --- Latency: 1960000 cycles; Energy: 305.15 uJ. Passing?
	True

[INFO] 2025-03-03 22:34:20,322 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


3.1: Optimized 4x4 --- Latency: 1960000 cycles; Energy: 262.89 uJ. Passing?
	True

3.1: Minimum energy achieved: 262.89 uJ. Passing?
	True


### Question 2
Manually generating the best mapping for each architecture and layer shape is rather time-consuming, even if the search is performed under a tightly constrained map sapace, *e.g.,* the one in question 2.2. Therefore, Timeloop provides automatic mapspace search functinality when appropriate map space constriants are given.

To perform an automatic mapspace search, you need to provide mapspace constraints as an input. Mapspace constraints specify the limitations imposed by your dataflow or hardware structures. They include the same directives as the mapping, except they are an *incomplete* description of the mapping, and they allow the mapper to optimize over all unspecified parts. An example mapspace constraint is shown below (`designs/system/constraints.yaml`). To automatically search the mapspace with the constraints file, you should run the `run_timeloop_mapper` command.

*The search should take less than 5 minutes to finish. If you are running the command from the shell instead of running the below cell, you can also terminate it whenever you want by pressing Ctrl+C (you will need to wait for Timeloop to finish the remainig computations after you send the signal; the terminated threads will have a dash next to its id).*

In [71]:
show_config('designs/system/constraints.yaml')

constraints:
  version: 0.4
  targets:
  - target: DRAM
    type: dataspace
    keep: [Inputs, Outputs, Weights]
    bypass: []
  - target: DRAM
    type: temporal
    factors: [R=1, S=1, P=1, Q=1]
    permutation: [R, S, Q, P, C, M, N]
  - target: global_buffer
    type: dataspace
    keep: [Inputs, Outputs, Weights]
    bypass: []
  - target: global_buffer
    type: temporal
    factors: [P=1, Q=1, R=1, S=1]
    permutation: [S, R, Q, P, C, M, N]
  - target: PE
    type: spatial
    factors: 
    - P=1
    - Q=1
    - R=1
    - S=1
    - N=1
    - C<={{pe_meshX}}
    - M<={{pe_meshY}}
    permutation: [C, M, R, S, P, Q, N]
    split: 1
  - target: scratchpad
    type: dataspace
    keep: [Weights]
    bypass: [Inputs, Outputs]
  - target: scratchpad
    type: temporal
    factors: [R=0, S=0, P=0, Q=0, M=1, C=1]
    permutation: [Q, P, N, C, M, S, R]

  - target: weight_reg
    type: dataspace
    keep: [Weights]
    bypass: [Inputs, Outputs]
  - target: weight_reg
    type: temporal


In [72]:
sys_1x16_result = run_timeloop_mapper(
    {'pe_meshX': 1, 'pe_meshY': 16},
    architecture='designs/system/arch.yaml',
    problem='layer_shapes/conv2.yaml',
    constraints='designs/system/constraints.yaml',
    mapper='designs/_include/mapper.yaml'
)

sys_1x16_stats = open('./output_dir/timeloop-mapper.stats.txt', 'r').read()
sys_1x16_mapping = sys_1x16_result.mapping

# Stats for the best mapping found by the mapper.
print(sys_1x16_result.cycles)

[INFO] 2025-03-03 22:34:21,810 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


3920000


In [73]:
lines = sys_1x16_stats.split('\n')
energy = float([l for l in lines if 'Energy:' in l][0].split(' ', 2)[1])
cycles = int([l for l in lines if 'Cycles:' in l][0].split(' ', 1)[1])
print(energy, cycles)

393.85 3920000


In [74]:
# Loop nest of the best mapping found by the mapper.
print(sys_1x16_mapping)

DRAM [ Weights:800 (800) Inputs:204800 (204800) Outputs:313600 (313600) ] 
-------------------------------------------------------------------------
| for N in [0:50)

global_buffer [ Weights:800 (800) Inputs:4096 (4096) Outputs:6272 (6272) ] 
--------------------------------------------------------------------------
|   for C in [0:4)

inter_PE_spatial [ ] 
--------------------
|     for M in [0:8) (Spatial-Y)

scratchpad [ Weights:25 (25) ] 
------------------------------
|       for R in [0:5)
|         for S in [0:5)
|           for P in [0:28)
|             for Q in [0:28)

weight_reg [ Weights:1 (1) ] 
input_activation_reg [ Inputs:1 (1) ] 
output_activation_reg [ Outputs:1 (1) ] 
---------------------------------------
|               << Compute >>



In this question, we have provided you with a much more relaxed constraint `designs/system/constraints_relaxed.yaml`. 
    
Please examine the constraints, and list two additional relaxations on the mapspace constraints in `designs/system/constraints_relaxed.yaml` comparing to `designs/system/constraints.yaml` (*Note: there are more than two relaxations, but you only need to list two*)

In [75]:
show_config('designs/system/constraints_relaxed.yaml')

constraints:
  version: 0.4
  targets:
  - target: DRAM
    type: dataspace
    keep: [Inputs, Outputs, Weights]
    bypass: []
  - target: global_buffer
    type: dataspace
    keep: [Inputs, Outputs, Weights]
    bypass: []
  - target: global_buffer
    type: temporal
    factors: [P=1, Q=1, R=1, S=1]
    permutation: [S, R, Q, P, C, M, N]
  - target: PE
    type: spatial
    factors: 
    - P=1
    - Q=1
    - R=1
    - S=1
    - N=1
    - C<={{pe_meshX}}
    - M<={{pe_meshY}}
    permutation: [C, M, R, S, P, Q, N]
    split: 1
  - target: scratchpad
    type: dataspace
    keep: [Weights]
    bypass: [Inputs, Outputs]
  - target: scratchpad
    type: temporal
    factors: [R=0, S=0, P=0, Q=0, C=1]
    permutation: [Q, P, S, R]

  - target: weight_reg
    type: dataspace
    keep: [Weights]
    bypass: [Inputs, Outputs]
  - target: weight_reg
    type: temporal
    factors: [R=1, S=1, P=1, Q=1, N=1, C=1, M=1]
    permutation: [R, S, P, Q, C, M, N]
  - target: input_activation_reg
  

In [76]:
answer(
    question='3.2',
    subquestion='List two hardware levels for which constraints have been relaxed. Answer as a Python list of strings (e.g., ["PE", "DRAM"]).',
    answer= ['DRAM', 'scratchpad'], # Answer here
    required_type=[str, str]
)

3.2: List two hardware levels for which constraints have been relaxed. Answer as a Python list of strings (e.g., ["PE", "DRAM"]).
	['DRAM', 'scratchpad']


Below, we run the mapper on all three architectures (1x16, 2x8, 4x4) on all three workloads (conv1, conv2, and fc1). For each workload, find the architecture that has the highest throughput by inspecting `all_stats`. If two architectures result in the same throughput, choose the one that's less energy consuming. Please fill in the chart below. 

In [77]:
workloads = {
    'conv1': 'layer_shapes/conv1.yaml',
    'conv2': 'layer_shapes/conv2.yaml',
    'fc1': 'layer_shapes/fc1.yaml'
}

pe_array_shapes = [
    (1, 16),
    (2, 8),
    (4, 4)
]

all_stats = {'conv1': {}, 'conv2': {}, 'fc1': {}}
all_maps = {'conv1': {}, 'conv2': {}, 'fc1': {}}

for name, workload in workloads.items():
    print('')
    for meshX, meshY in [(1, 16), (2, 8), (4, 4)]:
        result = run_timeloop_mapper(
            {'pe_meshX': meshX, 'pe_meshY': meshY},
            architecture='designs/system/arch.yaml',
            problem=workload,
            constraints='designs/system/constraints_relaxed.yaml',
            mapper='designs/_include/mapper.yaml',
            pe_spatial_c_constraint=True, # An extra constraint to speed up the mapper for this example.
        )
        stats = open('./output_dir/timeloop-mapper.stats.txt', 'r').read()
        mapping = result.mapping
        all_stats[name][(meshX, meshY)] = stats
        all_maps[name][(meshX, meshY)] = mapping
        lines = stats.split('\n')
        energy = float([l for l in lines if 'Energy:' in l][0].split(' ', 2)[1])
        cycles = int([l for l in lines if 'Cycles:' in l][0].split(' ', 1)[1])
        
        answer(
            question='3.2',
            subquestion=f'{meshX}x{meshY} cycles and energy (uJ) for {name}',
            answer=[cycles, energy],
            required_type=[int, Number]
        )


[INFO] 2025-03-03 22:34:23,594 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


3.2: 1x16 cycles and energy (uJ) for conv1
	[980000, 68.54]
[INFO] 2025-03-03 22:34:27,023 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


3.2: 2x8 cycles and energy (uJ) for conv1
	[980000, 68.54]
[INFO] 2025-03-03 22:34:30,380 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


...

3.2: 1x16 cycles and energy (uJ) for fc1
	[313600, 69.39]
[INFO] 2025-03-03 22:34:58,744 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


3.2: 2x8 cycles and energy (uJ) for fc1
	[313600, 55.62]
[INFO] 2025-03-03 22:35:03,434 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/lab3/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


3.2: 4x4 cycles and energy (uJ) for fc1
	[313600, 49.7]


Additionally, you will need to consider the workload shapes, which are printed below for your convenience:

In [78]:
for workload, file in workloads.items():
	print(f"### {workload} ###")
	print(open(file, 'r').read())

### conv1 ###
problem:
  version: 0.4
  shape:
    name: "CNN_Layer"
    dimensions: [ C, M, R, S, N, P, Q ]
    coefficients:
    - name: Wstride
      default: 1
    - name: Hstride
      default: 1
    - name: Wdilation
      default: 1
    - name: Hdilation
      default: 1

    data_spaces:
    - name: Weights
      projection:
      - [ [C] ]
      - [ [M] ]
      - [ [R] ]
      - [ [S] ]
    - name: Inputs
      projection:
      - [ [N] ]
      - [ [C] ]
      - [ [R, Wdilation], [P, Wstride] ] # SOP form: R*Wdilation + P*Wstride
      - [ [S, Hdilation], [Q, Hstride] ] # SOP form: S*Hdilation + Q*Hstride
    - name: Outputs
      projection:
      - [ [N] ]
      - [ [M] ]
      - [ [Q] ]
      - [ [P] ]
      read_write: True

  instance:
    C: 1  # inchn
    M: 4  # outchn
    R: 5   # filter height
    S: 5   # filter width
    P: 28  # ofmap height
    Q: 28  # ofmap width
    N: 50   # batch size
### conv2 ###
problem:
  version: 0.4
  shape:
    name: "CNN_Layer"
    d

In [79]:
for k, data in all_stats['conv1'].items(): 
    print(k)
    print(data)

'''
conv1
(1, 16)
Cycles: 980000
Energy: 68.54 uJ
Utilization: 25.00%

(2, 8)
Cycles: 980000
Energy: 68.54 uJ
Utilization: 25.00%

(4, 4)
Cycles: 980000
Energy: 68.54 uJ
Utilization: 25.00%
'''

(1, 16)
Buffer and Arithmetic Levels
----------------------------
Level 0
-------
=== mac ===

    SPECS
    -----
    Word bits             : 16
    Instances             : 16 (1*16)
    Compute energy        : 3.27 pJ

    STATS
    -----
    Utilized instances      : 4
    Computes (total)        : 3920000
    Cycles                  : 980000
    Energy (total)          : 12838000.00 pJ
    Area (total)            : 27624.00 um^2

Level 1
-------
=== output_activation_reg ===

    SPECS
    -----
        Technology                      : SRAM
        Size                            : 1
        Word bits                       : 16
        Block size                      : 1
        Cluster size                    : 1
        Instances                       : 16 (1*16)
        Shared bandwidth                : -
        Read bandwidth                  : -
        Write bandwidth                 : -
        Multiple buffering              : 1.00
        Effective size                  

'\nconv1\n(1, 16)\nCycles: 980000\nEnergy: 68.54 uJ\nUtilization: 25.00%\n\n(2, 8)\nCycles: 980000\nEnergy: 68.54 uJ\nUtilization: 25.00%\n\n(4, 4)\nCycles: 980000\nEnergy: 68.54 uJ\nUtilization: 25.00%\n'

In [80]:
for k, data in all_stats['fc1'].items(): 
    print(k)
    print(data)

# '''
# conv2
# (1, 16)
# Utilization: 50.00%
# Cycles: 3920000
# Energy: 393.85 uJ

# (2, 8)
# Utilization: 100.00%
# Cycles: 1960000
# Energy: 300.00 uJ

# (4, 4)
# Utilization: 100.00%
# Cycles: 1960000
# Energy: 262.76 uJ
    
# '''

# '''
# fc1: 
# (1, 16): 
# Utilization: 100.00%
# Cycles: 313600
# Energy: 69.39 uJ

# (2, 8)
# Utilization: 100.00%
# Cycles: 313600
# Energy: 55.62 uJ

# (4, 4)
# Utilization: 100.00%
# Cycles: 313600
# Energy: 49.70 uJ
    
# '''

full_data = {
    'conv1': {
        (1, 16): 980000, 
        (2, 8): 980000, 
        (4, 4): 980000
    }, 
    'conv2': {
        (1, 16): 3920000, 
        (2, 8): 1960000, 
        (4, 4): 1960000
    }, 
    'fc1': {
        (1, 16): 313600, 
        (2, 8): 313600, 
        (4, 4): 313600
    }
}


(1, 16)
Buffer and Arithmetic Levels
----------------------------
Level 0
-------
=== mac ===

    SPECS
    -----
    Word bits             : 16
    Instances             : 16 (1*16)
    Compute energy        : 3.27 pJ

    STATS
    -----
    Utilized instances      : 16
    Computes (total)        : 5017600
    Cycles                  : 313600
    Energy (total)          : 16432640.00 pJ
    Area (total)            : 27624.00 um^2

Level 1
-------
=== output_activation_reg ===

    SPECS
    -----
        Technology                      : SRAM
        Size                            : 1
        Word bits                       : 16
        Block size                      : 1
        Cluster size                    : 1
        Instances                       : 16 (1*16)
        Shared bandwidth                : -
        Read bandwidth                  : -
        Write bandwidth                 : -
        Multiple buffering              : 1.00
        Effective size                 

In [81]:
answer(
    question='3.2',
    subquestion='(True/False) There is a PE array shape that achieves fewer cycles and energy compared to other array shapes for conv1.',
    answer= False,
    required_type=bool
)
answer(
    question='3.2',
    subquestion='Given the array shapes (1x16, 2x8, and 4x4) and the workload shape of conv1, what is the maximum PE array utilization for each array shape? Answer as a list, e.g., [0.25, 0.5, 1].',
    answer= [0.25, 0.25, 0.25], # Answer here
    required_type=[Number, Number, Number]
)
answer(
    question='3.2',
    subquestion='Best [architecture name, cycles, total energy (uJ)] for conv2. Please answer as a list, e.g., ["2x8", 3920000, 68.54].',
    answer= ['4x4', 1960000, 262.76], # Answer here
    required_type=[('2x8', '1x16', '4x4'), int, Number]
)
answer(
    question='3.2',
    subquestion='Best [architecture name, cycles, total energy (uJ)] for fc1. Please answer as a list with 3 elements. Please answer as a list, e.g., ["2x8", 3920000, 68.54].',
    answer= ['4x4', 313600, 49.70], # Answer here
    required_type=[('2x8', '1x16', '4x4'), int, Number]
)

3.2: (True/False) There is a PE array shape that achieves fewer cycles and energy compared to other array shapes for conv1.
	False
3.2: Given the array shapes (1x16, 2x8, and 4x4) and the workload shape of conv1, what is the maximum PE array utilization for each array shape? Answer as a list, e.g., [0.25, 0.5, 1].
	[0.25, 0.25, 0.25]
3.2: Best [architecture name, cycles, total energy (uJ)] for conv2. Please answer as a list, e.g., ["2x8", 3920000, 68.54].
	['4x4', 1960000, 262.76]
3.2: Best [architecture name, cycles, total energy (uJ)] for fc1. Please answer as a list with 3 elements. Please answer as a list, e.g., ["2x8", 3920000, 68.54].
	['4x4', 313600, 49.7]


### Question 3
Your circuit designer has told you that it is too expensive to have a separate architecture for each layer shape. You must now have a fixed architecture (i.e. fixed height and width of the PE array). Based on this specific architecture, you can change the mapping according to different layer shapes. 

What is the best architecture that achieves the **highest average throughput (1/cycles)** of those three layer shapes among all the architectures explored in question 1? Calculate throughput for each layer independently, then average (DO NOT calculate 1 / (sum cycles)). Please fill in the chart below using that architecture.

In [82]:
for arch in all_stats['conv1'].keys(): 
    print(arch)
    th = 0
    for layer in all_stats.keys(): 
        print(layer)
        th += 1/full_data[layer][arch]
    print(th / 3)
    
    

(1, 16)
conv1
conv2
fc1
1.4880952380952381e-06
(2, 8)
conv1
conv2
fc1
1.5731292517006804e-06
(4, 4)
conv1
conv2
fc1
1.5731292517006804e-06


In [83]:
answer(
    question='3.3',
    subquestion='What is the best overall architecture for the three workloads?',
    answer= '4x4',
    required_type=('1x16', '2x8', '4x4')
)
answer(
    question='3.3',
    subquestion='What are the [cycles, total energy (uJ)] for this architecture and conv1?',
    answer= [980000, 68.54], # Answer here
    required_type=[int, Number]
)
answer(
    question='3.3',
    subquestion='What are the [cycles, total energy (uJ)] for this architecture and conv2?',
    answer= [1960000, 262.76], # Answer here
    required_type=[int, Number]
)
answer(
    question='3.3',
    subquestion='What are the [cycles, total energy (uJ)] for this architecture and fc1?',
    answer= [313600, 49.70], # Answer here
    required_type=[int, Number]
)

3.3: What is the best overall architecture for the three workloads?
	4x4
3.3: What are the [cycles, total energy (uJ)] for this architecture and conv1?
	[980000, 68.54]
3.3: What are the [cycles, total energy (uJ)] for this architecture and conv2?
	[1960000, 262.76]
3.3: What are the [cycles, total energy (uJ)] for this architecture and fc1?
	[313600, 49.7]
